In [11]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nameparser import HumanName

In [12]:
henry = pd.read_csv('Henry_Ford_Health_System_2020-01-06.csv')

In [13]:
len(henry)

7133

In [14]:
ppd = pd.read_csv('ppd_spec.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (14,27,28,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
# def clean_name(name):
#     name_split = name.split(',')
#     full_name = name_split[0].upper()
#     full_name_split = full_name.split(' ')
#     first_name = full_name_split[0]
# #     print(first_name)
#     last_name = full_name_split[-1]
# #     print(last_name)
#     full_name_split.remove(first_name)
#     middle_name = 'None'
#     if len(full_name_split) > 0:
#         full_name_split.remove(last_name)
#         if len(full_name_split)>0:
#             middle_name = full_name_split[0]
#             for word in full_name_split[1:]:
#                 middle_name = middle_name + ' ' + word
#     name_dict = {
#         'NAME':name,
#         'FIRST_NAME': first_name,
#         'LAST_NAME': last_name,
#         'MIDDLE_NAME': middle_name
#     }
#     return(name_dict)

def split_names(df):
    df = df.drop_duplicates()
    dict_list = []
    for row in df.itertuples():
        name_parsed = HumanName(row.NAME)
        name_dict = {
        'NAME':row.NAME,
        'FIRST_NAME': name_parsed.first.upper(),
        'LAST_NAME': name_parsed.last.upper(),
        'MIDDLE_NAME': name_parsed.middle.upper(),
        'SUFFIX':name_parsed.suffix.upper(),
        'NICKNAME':name_parsed.nickname.upper(),
        'TITLE':name_parsed.title.upper(),
        }
        dict_list.append(name_dict)
    name_df = pd.DataFrame(dict_list)
    new_df = pd.merge(name_df, df, on = 'NAME')
    return(new_df)

In [16]:
def match_ppd_name(ppd,df):
    count = len(df)
    matched = pd.merge(ppd, df, on = ['FIRST_NAME','LAST_NAME'], how = 'right')
    names = []
    duplicate_names = []
    missing_names =[]
    for row in matched.itertuples():
        if row.NAME not in names:
            names.append(row.NAME)
        elif row.NAME not in duplicate_names:
            duplicate_names.append(row.NAME)
        if pd.isna(row.ME)==True:
            missing_names.append(row.NAME)
    
    duplicates = matched[matched.NAME.isin(duplicate_names)]
    matched = matched.drop_duplicates('NAME', keep = False)
    
    print(f'{len(duplicates)} duplicates found.')
    print(f'{len(missing_names)} names missing.')
    
    return(duplicates, missing_names, matched)

In [17]:
def count_matches(matched_df):
    names = []
    duplicate_names = []
    for row in matched_df.itertuples():
        if row.NAME not in names:
            names.append(row.NAME)
        elif row.NAME not in duplicate_names:
            duplicate_names.append(row.NAME)
    
    duplicates = matched_df[matched_df.NAME.isin(duplicate_names)]
    matched = matched_df.drop_duplicates('NAME', keep = False)
    
    print(f'{len(duplicates)} duplicates found.')
    print(f'{len(matched)} unique matches found.')
    
    return(duplicates, matched)

In [18]:
def match_ppd_spec_fuzzy(matched_df):
    keep_list = []
    for row in matched_df.itertuples():
        keep = False
        if fuzz.ratio(row.SPECIALTY.upper(), row.DESC_PRIM) > 80:
            keep = True
        elif row.DESC_PRIM in row.SPECIALTY.upper():
            keep = True
        elif row.SPECIALTY.upper() in row.DESC_PRIM:
            keep = True
        keep_list.append(keep)
    matched_df['SPEC_MATCH']=keep_list
    matched_df = matched_df[matched_df['SPEC_MATCH']==True]
    return(matched_df)

In [19]:
def match_ppd_state(matched_df):
    state_match_list = []
    for row in matched_df.itertuples():
        state_match = False
        if row.POLO_STATE == row.LOCATION_ADDRESS_STATE:
            state_match = True
        state_match_list.append(state_match)
    matched_df['STATE_MATCH']=state_match_list
    matched_df = matched_df[matched_df['STATE_MATCH']==True]
    return(matched_df)
        
def match_ppd_city(matched_df):
    city_match_list = []
    for row in matched_df.itertuples():
        city_match = False
        if row.POLO_CITY == row.LOCATION_ADDRESS_CITY.upper():
            city_match = True
        city_match_list.append(city_match)
    matched_df['CITY_MATCH']=city_match_list
    matched_df = matched_df[matched_df['CITY_MATCH']==True]
    return(matched_df)

def match_ppd_spec_hard(matched_df):
    spec_match_list = []
    for row in matched_df.itertuples():
        spec_match = False
        if row.DESC_PRIM == row.SPECIALTY.upper():
            spec_match = True
        spec_match_list.append(spec_match)
    matched_df['SPEC_MATCH']=spec_match_list
    matched_df = matched_df[matched_df['SPEC_MATCH']==True]
    return(matched_df)

In [36]:
henry.drop_duplicates('NAME')

,NAME,SPECIALTY,SECONDARY_SPECIALTY,PHONE,FAX,LOCATION_NAME,LOCATION_ADDRESS,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_ZIP,DEPARTMENT,PROGRAM,PARENT_ORG_NAME
0,Saadia Abbas,Family Medicine,None,586-421-3052,None,Henry Ford Medical Center Sterling Heights,3500 15 Mile Rd,Sterling Heights,MI,48310,None,None,Henry Ford Health System
3,Sami Abbasi,Dermatology,None,734-675-0835,None,None,"133 W Main St, Ste 251",Northville,MI,48167,None,None,Henry Ford Health System
5,Tariq Abdelkarim,Pulmonary Disease,None,517-437-3879,None,Hillsdale Pulmonary Critical Care & Sleep Medi...,3271 W Carleton Road,Hillsdale,MI,49242,None,None,Henry Ford Health System
7,Firas Abdollah,Urology,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System
9,Mayada Abdul-Aziz,Internal Medicine,None,800-436-7936,None,Henry Ford Medical Center - Taylor,24555 Haig Rd,Taylor,MI,48180,None,None,Henry Ford Health System
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7125,Ashley Zuzelski,Emergency Medicine,None,517-205-4811,None,Henry Ford Allegiance Emergency Department,205 N East Ave,Jackson,MI,49201,None,None,Henry Ford Health System
7126,Dace Zvirbulis,Neurology,None,800-436-7936,None,Henry Ford Hospital,2799 W Grand Blvd,Detroit,MI,48202,None,None,Henry Ford Health System
7129,Raimonds Zvirbulis,Orthopedic Surgery,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System
7131,Bryan Zweig,Cardiology,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System


In [21]:
henry_df = split_names(henry)
henry_df = henry_df.drop_duplicates('NAME')

In [34]:
henry_df.head()

,NAME,FIRST_NAME,LAST_NAME,MIDDLE_NAME,SUFFIX,NICKNAME,TITLE,SPECIALTY,SECONDARY_SPECIALTY,PHONE,FAX,LOCATION_NAME,LOCATION_ADDRESS,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_ZIP,DEPARTMENT,PROGRAM,PARENT_ORG_NAME
0,Saadia Abbas,SAADIA,ABBAS,,,,,Family Medicine,None,586-421-3052,None,Henry Ford Medical Center Sterling Heights,3500 15 Mile Rd,Sterling Heights,MI,48310,None,None,Henry Ford Health System
9,Sami Abbasi,SAMI,ABBASI,,,,,Dermatology,None,734-675-0835,None,None,"133 W Main St, Ste 251",Northville,MI,48167,None,None,Henry Ford Health System
13,Tariq Abdelkarim,TARIQ,ABDELKARIM,,,,,Pulmonary Disease,None,517-437-3879,None,Hillsdale Pulmonary Critical Care & Sleep Medi...,3271 W Carleton Road,Hillsdale,MI,49242,None,None,Henry Ford Health System
17,Firas Abdollah,FIRAS,ABDOLLAH,,,,,Urology,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System
21,Mayada Abdul-Aziz,MAYADA,ABDUL-AZIZ,,,,,Internal Medicine,None,800-436-7936,None,Henry Ford Medical Center - Taylor,24555 Haig Rd,Taylor,MI,48180,None,None,Henry Ford Health System


In [23]:
specs = list(henry_df['SPECIALTY'].unique())
bad_spec = ['Physician Assistant',
 'Nurse Practitioner',
 'Anesthesia - CRNA',
 'Midwife',
 'Anesthesia - CRNA, Nurse Practitioner',
 'Chiropractor',
 'Psychologist',
 'Optometrist',
 'Pediatric Dentistry',
 'Veterinarian',
 'Midwife, Nurse Practitioner']
specs = [x for x in specs if x not in bad_spec]

henry_df = henry_df[henry_df['SPECIALTY'].isin(specs)]

In [37]:
len(henry_df)

3104

In [25]:
duplicate_names_henry, missing_names_henry, matched_henry = match_ppd_name(ppd, henry_df) 3104

4454 duplicates found.
528 names missing.


In [26]:
new_matched = match_ppd_spec_fuzzy(duplicate_names_henry)
new_matched.head()
new_duplicates, new_matches = count_matches(new_matched)
new_matches.head()
print(len(matched_henry))
matched_henry = matched_henry.append(new_matches)
print(len(matched_henry))
print(f'{len(new_matches)} new matches confirmed. {len(matched_henry)} total matches confirmed.')

new_matched_2 = match_ppd_state(new_duplicates)
new_duplicates_2, new_matches_2 = count_matches(new_matched_2)
print(len(matched_henry))
matched_henry = matched_henry.append(new_matches_2)
print(len(matched_henry))
print(f'{len(new_matches_2)} new matches confirmed. {len(matched_henry)} total matches confirmed.')

new_matched_3 = match_ppd_city(new_duplicates_2)
new_duplicates_3, new_matches_3 = count_matches(new_matched_3)
matched_henry = matched_henry.append(new_matches_3)
print(f'{len(new_matches_3)} new matches confirmed. {len(matched_henry)} total matches confirmed.')

new_matched_4 = match_ppd_spec_hard(new_duplicates_3)
new_duplicates_4, new_matches_4 = count_matches(new_matched_4)
matched_henry = matched_henry.append(new_matches_4)
print(f'{len(new_matches_4)} new matches confirmed. {len(matched_henry)} total matches confirmed.')

302 duplicates found.
328 unique matches found.
2512
2840
328 new matches confirmed. 2840 total matches confirmed.
22 duplicates found.
86 unique matches found.
2840
2926
86 new matches confirmed. 2926 total matches confirmed.
0 duplicates found.
0 unique matches found.
0 new matches confirmed. 2926 total matches confirmed.
0 duplicates found.
0 unique matches found.
0 new matches confirmed. 2926 total matches confirmed.


C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [27]:
new_duplicates_2

,SPEC_CD_SEC,DESC_SEC,SPEC_CD_PRIM,DESC_PRIM,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME_x,SUFFIX_x,...,LOCATION_ADDRESS,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_ZIP,DEPARTMENT,PROGRAM,PARENT_ORG_NAME,SPEC_MATCH,STATE_MATCH,CITY_MATCH
216,SMI,SLEEP MEDICINE (INTERNAL MEDICINE),IM,INTERNAL MEDICINE,4.950375e+10,ASHOK K GUPTA MD,GUPTA,ASHOK,KUMAR,NaN,...,5500 Auto Club Dr,Dearborn,MI,48126,None,None,Henry Ford Health System,True,True,False
217,CD,CARDIOVASCULAR DISEASE,IM,INTERNAL MEDICINE,4.950374e+10,ASHOK K GUPTA MD,GUPTA,ASHOK,KUMAR,NaN,...,5500 Auto Club Dr,Dearborn,MI,48126,None,None,Henry Ford Health System,True,True,False
218,FM,FAMILY MEDICINE,IM,INTERNAL MEDICINE,4.952972e+10,ASHOK K GUPTA MD,GUPTA,ASHOK,K,NaN,...,5500 Auto Club Dr,Dearborn,MI,48126,None,None,Henry Ford Health System,True,True,False
222,US,UNSPECIFIED,IM,INTERNAL MEDICINE,4.952075e+10,ASHOK GUPTA MD,GUPTA,ASHOK,NaN,NaN,...,5500 Auto Club Dr,Dearborn,MI,48126,None,None,Henry Ford Health System,True,True,False
1414,CD,CARDIOVASCULAR DISEASE,IM,INTERNAL MEDICINE,4.952170e+10,SYED A JAFRI MD,JAFRI,SYED,ABBAS HASAN,NaN,...,14500 Hall Rd,Sterling Hgts,MI,48313,None,None,Henry Ford Health System,True,True,False
1428,US,UNSPECIFIED,IM,INTERNAL MEDICINE,2.576151e+09,SYED S JAFRI DO,JAFRI,SYED,SADIQ MOHAMMED,NaN,...,14500 Hall Rd,Sterling Hgts,MI,48313,None,None,Henry Ford Health System,True,True,False
2012,EM,EMERGENCY MEDICINE,FM,FAMILY MEDICINE,2.507782e+09,BRADFORD J WOELKE MD,WOELKE,BRADFORD,JAMES,NaN,...,6620 Highland Rd,Waterford,MI,48327,None,None,Henry Ford Health System,True,True,False
2013,US,UNSPECIFIED,FP,FAMILY MEDICINE,2.507171e+09,BRADFORD J WOELKE MD,WOELKE,BRADFORD,JAMES,NaN,...,6620 Highland Rd,Waterford,MI,48327,None,None,Henry Ford Health System,True,True,False
2648,US,UNSPECIFIED,FM,FAMILY MEDICINE,2.507100e+09,CHERYL L SMITH MD,SMITH,CHERYL,LEE,NaN,...,19020 Fort Street,Riverview,MI,48193,None,None,Henry Ford Health System,True,True,False
2649,US,UNSPECIFIED,FM,FAMILY MEDICINE,2.576981e+09,CHERYL L SMITH DO,SMITH,CHERYL,LEE,NaN,...,19020 Fort Street,Riverview,MI,48193,None,None,Henry Ford Health System,True,True,False


In [28]:
missing_df = henry_df[henry_df['NAME'].isin(missing_names_henry)]

In [29]:
add_matches = []
for name in missing_names_henry:
    if name != 'None':
        first_match = ppd[ppd['FIRST_NAME'] == name.upper().split(' ')[0]]
        last_match = ppd[ppd['LAST_NAME'] == name.upper().split(' ')[1]]
        if len(first_match) == 1:
            print(f'{name} : {list(first_match["MAILING_NAME"])[0]}')
            new_dict ={'MAILING_NAME': list(first_match['MAILING_NAME'])[0],
                       'ME': list(first_match['ME'])[0],
                       'DESC_PRIM': list(first_match['DESC_PRIM'])[0],
                       'NAME':name
            }
            add_matches.append(new_dict)
        if len(last_match) == 1:
            print(f'{name} : {list(last_match["MAILING_NAME"])[0]}')
            new_dict ={'MAILING_NAME': list(last_match['MAILING_NAME'])[0],
                       'ME': list(last_match['ME'])[0],
                       'DESC_PRIM': list(last_match['DESC_PRIM'])[0],
                       'NAME':name
            }
            add_matches.append(new_dict)

Muwaffak Abdulhak : MUWAFFAK M ABDUL-HAK MD
Odaliz Abreu Lanfranco : ODALIZ E ABREU LANFRANCO MD
Kheir Al-Zouhayli : M K AL-ZOUHAYLI MD
Salah Alenzi : BADER A ALENZI MD
Wamidh Alkhoory : WAMIDH L ALKHURRI MD
Lisa Allenspach : ERIC J ALLENSPACH MD
Mohamed-Rida Alsaden : MUHAMADRIDA A ALSADEN MD
Karthikeyan Ananthasubramaniam : K ANANTHASUBRAMANIAM MD
Graham Appleford : COLIN APPLEFORD DO
Samah Arsanious : DAVID E ARSANIOUS MD
Gregory Auner : JOHN D AUNER MD
Fatima Awarke : FATME AWARKE DO
Brian Bismack : GREGORY T BISMACK MD
Abby Bonato : ALEXANDER E BONATO MD
Jieli Chen : JIELI LI MD
Laura Dalla Vecchia : KUMAR P DALLA MD
Sairia Dass Ramcharan : SAIRIA DASS MD
Janice Demaray : MICHAEL J DEMARAY MD
Elaine Dziemianowicz : MARK S DZIEMIANOWICZ MD
Samar El-Achkar : TAREK M EL-ACHKAR MD
Stephanie Franso : DEANE FRANSO MD
Stephen Frascone : RALPH J FRASCONE MD
Shira Gordinier : MARY E GORDINIER MD
Gurpriya Gupta : GURPRIYA K SETHI MD
Xiaoxia Han : XIAOXIA ZHANG MD
Paul Heilborn : PAUL-DAVID 

KeyboardInterrupt: 

In [ ]:
attempts = pd.DataFrame(add_matches)
TEST = pd.merge(attempts, henry_df, on = 'NAME')
scores =[]
for row in TEST.itertuples():
    score =fuzz.ratio(row.MAILING_NAME,row.NAME.upper())
    scores.append(score)
TEST['SCORE']=scores

In [ ]:
TEST[TEST['SCORE']>72]

In [ ]:
first_df.to_csv('inf_test.csv')

In [ ]:
for doc in missing_names_henry:
    matched = False
    for NAME in names:
        if fuzz.ratio(NAME, doc.upper())>70:
            potential_list.append(NAME)
            print(f'{NAME} : {doc}')
            matched = True
            if fuzz.ratio(NAME, doc.upper())>84:
                break
    if matched == False:
        print(f'{doc} not found')

In [ ]:
names = list(ppd['MAILING_NAME'])

In [ ]:
fuzz.ratio('MUWAFFAK M ABDUL-HAK MD','MUWAFFAK ABDULHAK')

In [ ]:
missing_names_henry[9:].sort()

In [ ]:
missing_names_henry.sort()

In [ ]:
names.sort()

In [ ]:
pd.merge(first_df, henry_df, on='LAST_NAME')[['NAME','MAILING_NAME']].head(50)

In [ ]:
ppd.dtypes

In [ ]:
missing_df.groupby('SPECIALTY').count().sort_values(by = 'NAME',ascending = False).head(50)

In [ ]:
missing = missing_df[missing_df['SPECIALTY'].isin(specs)]

In [ ]:
specs.remove(bad_spec)

In [ ]:
specs = [x for x in specs if x not in bad_spec]

In [ ]:
def look_for_doctor(name):
    potential_list =[]
    name_scores = []
    matched = False
    for NAME in names:
        score = fuzz.ratio(NAME, name.upper())
        if score > 70:
            potential_list.append(NAME)
            name_scores.append(score)
            print(f'{NAME} : {name}, {score}')
            matched = True
            if score >84:
                break
    if matched == False:
        print(f'{name} not found')
    return(potential_list, name_scores)

def match_doctor(name, specialty):
    potential_list, name_scores = look_for_doctor(name)
    if len(potential_list)==0:
        return([])
    filtered_ppd = ppd[ppd['MAILING_NAME'].isin(potential_list)]
    spec_scores = []
    for row in filtered_ppd.itertuples():
        spec_score = fuzz.ratio(row.DESC_PRIM, specialty.upper())
        spec_scores.append(spec_score)
    filtered_ppd['SPEC_SCORE']=spec_scores
    filtered_ppd['NAME_SCORE']=name_scores
    filtered_ppd['SCORE']=filtered_ppd['SPEC_SCORE']+filtered_ppd['NAME_SCORE']
    return(filtered_ppd)

def pick_match(name, specialty):
    filtered_ppd = match_doctor(name, specialty)
    if len(filtered_ppd)==0:
        return([])
    return(filtered_ppd[filtered_ppd.SPEC_SCORE == filtered_ppd.SPEC_SCORE.max()])

In [ ]:
potential_finds = pick_match('JOELLE ABOOD', 'Obstetrics & Gynecology')

In [ ]:
potential_finds = ppd[ppd['MAILING_NAME']=='JOELLE AOUN MD']

In [ ]:
j_list = ['JOEL E ABBOTT DO',
 'JOEL E ARROYO MD',
 'JOEL E BORKOW MD',
 'JOEL E BROOK MD',
 'JOELLE AOUN MD',
 'JOELLE MORROW MD',
 'JOELLEN BROWN MD',
 'NOELLE BOCTOR MD']
for thing in j_list:
    print(thing)
    print(fuzz.ratio('JOELLE ABOOD',thing))
    print('')

In [ ]:
fuzz.ratio('JOELLE ABOOD', 'JOELLE MORROW MD')

In [ ]:
ppd = ppd.sort_values('MAILING_NAME')

In [ ]:
num = 144
for row in missing[144:].itertuples():
    print(num)
    matches = pick_match(row.NAME, row.SPECIALTY)
    if len(matches)!=0:
        potential_finds.append(matches)
    num += 1

In [ ]:
missing[144:150]

In [ ]:
dict_list =[]
duplicate_df = henry_df[henry_df['NAME'].isin(duplicate_names_henry)]
duplicate_complete = pd.merge(duplicate_df, ppd, on=['FIRST_NAME','LAST_NAME'])
for row in duplicate_complete.itertuples():
    if row.SPECIALTY in specs:
        print(f'{row.NAME} : {row.MAILING_NAME}\n{row.SPECIALTY} : {row.DESC_PRIM}')
        print('')
        keep = False
        if fuzz.ratio(row.SPECIALTY.upper(), row.DESC_PRIM) > 80:
            keep = True
        elif row.DESC_PRIM in row.SPECIALTY.upper():
            keep = True
        elif row.SPECIALTY.upper() in row.DESC_PRIM:
            keep = True
        if keep == True:
            new_dict = {
                    'NAME': row.NAME,
                    'MAILING_NAME':row.MAILING_NAME,
                    'ME': row.ME
                }
            dict_list.append(new_dict)

In [ ]:
duplicate_complete['DESC_PRIM']

In [ ]:
len(pd.DataFrame(dict_list).drop_duplicates('NAME',keep=False))

In [ ]:
len(henry_df[henry_df['SPECIALTY'].isin(specs)])

In [ ]:
len(matched_henry[matched_henry['SPECIALTY'].isin(specs)].dropna(subset = ['ME']).drop_duplicates('NAME',keep=False)[['DESC_PRIM','SPECIALTY']])

In [ ]:
len(missing)

In [ ]:
'226 duplicates'

In [ ]:
potential_finds

In [ ]:
spec_name_matches = pd.DataFrame(dict_list)

In [ ]:
spec_name_matches_ = pd.merge(spec_name_matches, henry_df, on='NAME')
spec_name_matches_ = pd.merge(spec_name_matches_, ppd, on='ME')

In [ ]:
for col in spec_name_matches_.columns:
    print(col)

In [ ]:
state_match_list = []
for row in spec_name_matches_.itertuples():
    state_match = False
    if row.POLO_STATE == row.LOCATION_ADDRESS_STATE:
        state_match = True
    state_match_list.append(state_match)

In [ ]:
spec_name_matches_['STATE_MATCH']=state_match_list

In [ ]:
spec_name_matches_[spec_name_matches_['STATE_MATCH']==True][['SPECIALTY','DESC_PRIM','NAME','MAILING_NAME_x', 'POLO_CITY','LOCATION_ADDRESS_CITY']][350:400]

In [ ]:
2+2

In [ ]:
matched_henry

In [ ]:
matched_henry.dropna(subset = ['ME'])

In [ ]:
len(henry_df)

In [ ]:
for col in henry_df.columns:
    print(col)

In [ ]:
all_henry = pd.merge(henry_df, matched_henry, on = ['NAME',
'FIRST_NAME',
'LAST_NAME',
'NICKNAME',
'TITLE',
'SPECIALTY',
'SECONDARY_SPECIALTY',
'PHONE',
'FAX',
'LOCATION_NAME',
'LOCATION_ADDRESS',
'LOCATION_ADDRESS_CITY',
'LOCATION_ADDRESS_STATE',
'LOCATION_ADDRESS_ZIP',
'DEPARTMENT',
'PROGRAM',
'PARENT_ORG_NAME'], how = 'left')

In [ ]:
len(henry_df)

In [ ]:
still_na = all_henry[all_henry['ME'].isna()]

In [ ]:
def match_cleaned_last(still_na,ppd):
    cleaned_lasts_2 = []
    for last in list(still_na['LAST_NAME']):
        new_last = str(last).replace(' ','').replace(' ','').replace('-','').replace("'","")
        cleaned_lasts_2.append(new_last)
    still_na['CLEANED_LAST']=cleaned_lasts_2
    cleaned_lasts = []
    for last in list(ppd['LAST_NAME']):
        new_last = str(last).replace(' ','').replace(' ','').replace('-','').replace("'","")
        cleaned_lasts.append(new_last)
    ppd['CLEANED_LAST']=cleaned_lasts
    new_matches = pd.merge(still_na,ppd,on=['FIRST_NAME','CLEANED_LAST'])
    return(new_matches)

In [ ]:
cleaned_matches = match_cleaned_last(still_na, ppd)

In [ ]:
clean_duplicates, clean_matches = count_matches(cleaned_matches)

In [ ]:
matched_henry.append(clean_matches)

In [ ]:
ppd_last_names=list(ppd['LAST_NAME'])

In [ ]:
ppd_last_names

In [ ]:
cleaned_lasts_2 = []
for last in list(still_na['LAST_NAME']):
#     if "'" in str(last):
#         print(last)
    new_last = str(last).replace(' ','').replace(' ','').replace('-','').replace("'","")
    cleaned_lasts_2.append(new_last)

In [ ]:
for row in still_na.itertuples():
    if " " in row.LAST_NAME:
        print(row.LAST_NAME)

In [ ]:
ppd['CLEANED_LAST']=cleaned_lasts

In [ ]:
still_na['CLEANED_LAST']=cleaned_lasts_2

In [ ]:
len(still_na)

In [ ]:
clean_duplicates.drop_duplicates('NAME')

In [ ]:
clean_matches

In [ ]:
still_na.drop_duplicates('NAME')

In [ ]:
missing_list = []
COUNT = 0
for row in still_na.itertuples():
    if row.NAME not in founded_names:
        COUNT += 1
        print(row.NAME)
        print(row.SPECIALTY)
        print(row.LOCATION_ADDRESS_CITY)
        print(row.LOCATION_ADDRESS_STATE)
        new_dic  = {
            'NAME':row.NAME,
            'FIRST_NAME':row.FIRST_NAME,
            'MIDDLE_NAME': row.MIDDLE_NAME,
            'LAST_NAME':row.LAST_NAME,
            'SPEC':row.SPECIALTY,
            'CITY':row.LOCATION_ADDRESS_CITY,
            'STATE':row.LOCATION_ADDRESS_STATE
        }
        missing_list.append(new_dic)
print(COUNT)
MISSING_DF = pd.DataFrame(missing_list)

In [ ]:
founded_names = list(cleaned_matches['NAME'])

In [ ]:
ppd[ppd['FIRST_NAME']=='SHANA'][0:51]

In [ ]:
ppd

In [ ]:
first_match = pd.merge(MISSING_DF, ppd, on = 'FIRST_NAME')

In [ ]:
first_match = first_match.rename(columns={'SPEC':'SPECIALTY'})

In [ ]:
spec_first = match_ppd_spec_fuzzy(first_match)

In [ ]:
spec_first_duplicates, spec_first_matches = count_matches(spec_first)

In [ ]:
for row in MISSING_DF.itertuples():
    pd.merge(MISSING_DF, ppd, on=)

In [ ]:
fuzz.ratio('LANFRANCO','ABREAU LANFRANCO')

In [ ]:
spec_first_matches[['NAME','LAST_NAME_x','LAST_NAME_y']]

In [ ]:
def match_check(thing_1, thing_2):
    match = False
    if str(thing_1).upper() == str(thing_2).upper():
        match = True
    if str(thing_2).upper() in str(thing_1).upper():
        match =True
    if fuzz.ratio(str(thing_1).upper(), str(thing_2).upper())>70:
        match =True
    return(match)

In [ ]:
def check_first_matches(MISSING_DF):
    me_name_list =[]
    for row in MISSING_DF.itertuples():
        first_match = ppd[ppd['FIRST_NAME']==row.FIRST_NAME]
        if len(first_match)==1:
            me_name = {'NAME':row.NAME,
                       'ME': list(first_match['ME'])[0]
                        }
            if match_check(list(first_match['DESC_PRIM'])[0],row.SPEC)==True:
                if match_check(list(first_match['LAST_NAME'])[0],row.LAST_NAME)==True:
                    me_name_list.append(me_name)

        else:
            for line in first_match.itertuples():
                if match_check(line.DESC_PRIM,row.SPEC)==True:
                    if match_check(line.LAST_NAME,row.LAST_NAME)==True:
                        me_name = {'NAME':row.NAME,
                       'ME': line.ME
                        }
                        me_name_list.append(me_name)
    

In [ ]:
MISSING_DF

In [ ]:
match_check('ABREU LANFRANCO','LANFRANCO')

In [ ]:
ppd[ppd['FIRST_NAME']=='ODALIZ']

In [ ]:
me_name_list

In [31]:
matched_henry.dropna(subset = ['ME']).to_csv('Matched_Henry.csv',index=False)

In [42]:
matched_henry[['PHONE','TELEPHONE_NUMBER']]

,PHONE,TELEPHONE_NUMBER
0,800-436-7936,3.139163e+09
139,586-263-2371,5.862632e+09
173,586-726-4823,5.867265e+09
214,517-205-7525,5.172051e+09
215,800-436-7936,7.349367e+09
...,...,...
4921,248-880-0123,2.488800e+09
5346,517-205-7060,NaN
5358,800-436-7936,2.486669e+09
5459,586-726-0044,5.867260e+09


In [41]:
for col in matched_henry.columns:
    print(col)

BIRTH_CITY
BIRTH_COUNTRY
BIRTH_STATE
BIRTH_YEAR
CITY
CITY_MATCH
DEPARTMENT
DESC_PRIM
DESC_SEC
FAX
FAX_NUMBER
FIRST_NAME
GME_INST_ID
GME_INST_STATE
GME_SPEC_1
GME_SPEC_2
LAST_NAME
LOCATION_ADDRESS
LOCATION_ADDRESS_CITY
LOCATION_ADDRESS_STATE
LOCATION_ADDRESS_ZIP
LOCATION_NAME
MAILING_LINE_1
MAILING_LINE_2
MAILING_NAME
MD_DO_CODE
ME
MEDSCHOOL_GRAD_YEAR
MEDSCHOOL_ID
MEDSCHOOL_STATE
MIDDLE_NAME_x
MIDDLE_NAME_y
MOST_RECENT_FORMER_FIRST_NAME
MOST_RECENT_FORMER_LAST_NAME
MOST_RECENT_FORMER_MIDDLE_NAME
NAME
NEXT_MOST_RECENT_FORMER_FIRST
NEXT_MOST_RECENT_FORMER_LAST
NEXT_MOST_RECENT_FORMER_MIDDLE
NICKNAME
PARENT_ORG_NAME
PHONE
POLO_CITY
POLO_MAILING_LINE_1
POLO_MAILING_LINE_2
POLO_STATE
POLO_ZIP
POST_GRADUATE_YEAR
PRIM_SPEC_CD
PROGRAM
SECONDARY_SPECIALTY
SEC_SPEC_CD
SPECIALTY
SPEC_CD_PRIM
SPEC_CD_SEC
SPEC_MATCH
STATE
STATE_MATCH
SUFFIX_x
SUFFIX_y
TELEPHONE_NUMBER
TITLE
TOP_CD
TRAINING_TYPE
YEAR_IN_PROGRAM
ZIP


In [45]:
matched_henry.dropna(subset=['ME'])

,BIRTH_CITY,BIRTH_COUNTRY,BIRTH_STATE,BIRTH_YEAR,CITY,CITY_MATCH,DEPARTMENT,DESC_PRIM,DESC_SEC,FAX,...,STATE,STATE_MATCH,SUFFIX_x,SUFFIX_y,TELEPHONE_NUMBER,TITLE,TOP_CD,TRAINING_TYPE,YEAR_IN_PROGRAM,ZIP
0,NaN,NaN,NaN,1976.0,DEARBORN HTS,NaN,None,CLINICAL NEUROPHYSIOLOGY,NEUROMUSCULAR MEDICINE (NEUROLOGY),None,...,MI,NaN,NaN,,3.139163e+09,,20.0,NaN,NaN,48127
139,DETROIT,US1,MI,1958.0,WASHINGTON,NaN,None,ANESTHESIOLOGY,PAIN MANAGEMENT,None,...,MI,NaN,NaN,,5.862632e+09,,20.0,NaN,NaN,48094
173,DETROIT,US1,MI,1968.0,ROCHESTER HLS,NaN,None,FAMILY MEDICINE,PAIN MANAGEMENT,None,...,MI,NaN,NaN,,5.867265e+09,,20.0,NaN,NaN,48309
214,HYDERABAD,IN1,NaN,1967.0,JACKSON,NaN,None,PHYSICAL MEDICINE AND REHABILITATION,PAIN MANAGEMENT,None,...,MI,NaN,NaN,,5.172051e+09,,20.0,NaN,NaN,49201
215,MAYFIELD HEIGHT,US1,OH,1981.0,ANN ARBOR,NaN,None,PAIN MEDICINE,PAIN MANAGEMENT,None,...,MI,NaN,NaN,,7.349367e+09,,20.0,NaN,NaN,48108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4921,NaN,IZ1,NaN,1980.0,BLOOMFLD HLS,NaN,None,FAMILY MEDICINE,UNSPECIFIED,None,...,MI,True,NaN,,2.488800e+09,,20.0,NaN,NaN,48302
5346,STAUNTON,US1,VA,1972.0,JACKSON,NaN,None,INTERNAL MEDICINE,UNSPECIFIED,None,...,MI,True,NaN,,NaN,,20.0,NaN,NaN,49201
5358,ADRIAN,US1,MI,1951.0,WATERFORD,NaN,None,INTERNAL MEDICINE,UNSPECIFIED,None,...,MI,True,NaN,,2.486669e+09,,20.0,NaN,NaN,48327
5459,NaN,IN1,NaN,1964.0,STERLING HTS,NaN,None,INTERNAL MEDICINE,UNSPECIFIED,None,...,MI,True,NaN,,5.867260e+09,,20.0,NaN,NaN,48313


In [46]:
henry

,NAME,SPECIALTY,SECONDARY_SPECIALTY,PHONE,FAX,LOCATION_NAME,LOCATION_ADDRESS,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_ZIP,DEPARTMENT,PROGRAM,PARENT_ORG_NAME
0,Saadia Abbas,Family Medicine,None,586-421-3052,None,Henry Ford Medical Center Sterling Heights,3500 15 Mile Rd,Sterling Heights,MI,48310,None,None,Henry Ford Health System
1,Saadia Abbas,Family Medicine,None,586-421-3052,None,HFM Urgent Care - Bruce Twp,80650 Van Dyke,Bruce Township,MI,48065,None,None,Henry Ford Health System
2,Saadia Abbas,Family Medicine,None,586-421-3052,None,HFM Urgent Care - Chesterfield,30795 23 Mile Rd,Chesterfield,MI,48047,None,None,Henry Ford Health System
3,Sami Abbasi,Dermatology,None,734-675-0835,None,None,"133 W Main St, Ste 251",Northville,MI,48167,None,None,Henry Ford Health System
4,Sami Abbasi,Dermatology,None,734-675-0835,None,Abbasi Dermatology,21401 Allen Road,Woodhaven,MI,48183,None,None,Henry Ford Health System
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7128,Dace Zvirbulis,Neurology,None,800-436-7936,None,Henry Ford Medical Center - Farmington Road,6530 Farmington Rd,West Bloomfield,MI,48322,None,None,Henry Ford Health System
7129,Raimonds Zvirbulis,Orthopedic Surgery,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System
7130,Raimonds Zvirbulis,Orthopedic Surgery,None,800-436-7936,None,Henry Ford West Bloomfield Hospital,6777 W Maple Rd,West Bloomfield,MI,48322,None,None,Henry Ford Health System
7131,Bryan Zweig,Cardiology,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System


In [58]:
mathced_test = matched_henry[['NAME','ME','TELEPHONE_NUMBER','PHONE']]

In [83]:
pd.merge(henry, SIMPLE, on='NAME', how = 'left')

,NAME,SPECIALTY,SECONDARY_SPECIALTY,PHONE,FAX,LOCATION_NAME,LOCATION_ADDRESS,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_ZIP,DEPARTMENT,PROGRAM,PARENT_ORG_NAME,ME
0,Saadia Abbas,Family Medicine,None,586-421-3052,None,Henry Ford Medical Center Sterling Heights,3500 15 Mile Rd,Sterling Heights,MI,48310,None,None,Henry Ford Health System,7.040297e+10
1,Saadia Abbas,Family Medicine,None,586-421-3052,None,HFM Urgent Care - Bruce Twp,80650 Van Dyke,Bruce Township,MI,48065,None,None,Henry Ford Health System,7.040297e+10
2,Saadia Abbas,Family Medicine,None,586-421-3052,None,HFM Urgent Care - Chesterfield,30795 23 Mile Rd,Chesterfield,MI,48047,None,None,Henry Ford Health System,7.040297e+10
3,Sami Abbasi,Dermatology,None,734-675-0835,None,None,"133 W Main St, Ste 251",Northville,MI,48167,None,None,Henry Ford Health System,2.576010e+09
4,Sami Abbasi,Dermatology,None,734-675-0835,None,Abbasi Dermatology,21401 Allen Road,Woodhaven,MI,48183,None,None,Henry Ford Health System,2.576010e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7128,Dace Zvirbulis,Neurology,None,800-436-7936,None,Henry Ford Medical Center - Farmington Road,6530 Farmington Rd,West Bloomfield,MI,48322,None,None,Henry Ford Health System,9.131694e+10
7129,Raimonds Zvirbulis,Orthopedic Surgery,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System,2.501712e+09
7130,Raimonds Zvirbulis,Orthopedic Surgery,None,800-436-7936,None,Henry Ford West Bloomfield Hospital,6777 W Maple Rd,West Bloomfield,MI,48322,None,None,Henry Ford Health System,2.501712e+09
7131,Bryan Zweig,Cardiology,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System,1.720042e+09


,NAME,ME,TELEPHONE_NUMBER,PHONE
0,Youssef Dakka,6.610204e+10,3.139163e+09,800-436-7936
139,Robert Peleman,2.507842e+09,5.862632e+09,586-263-2371
173,Theodore Tangalos,2.507942e+09,5.867265e+09,586-726-4823
214,Anandeep Kumar,4.957091e+10,5.172051e+09,517-205-7525
215,Sarah Money,3.843081e+09,7.349367e+09,800-436-7936
...,...,...,...,...
4921,Fadi Ibrahim,3.050108e+10,2.488800e+09,248-880-0123
5346,Richard Santos,1.803980e+09,NaN,517-205-7060
5358,Christine Meyer,2.501831e+09,2.486669e+09,800-436-7936
5459,William Massey,4.950890e+10,5.867260e+09,586-726-0044


In [81]:
new_phone_count =0
no_count =0
yes_count =0
for row in mathced_test.itertuples():
    if np.isnan(row.TELEPHONE_NUMBER)==False:
        if str(int(row.TELEPHONE_NUMBER)) == str(row.PHONE).replace('-',''):
            yes_count +=1
        else:
            no_count += 1
    elif row.PHONE != 'None':
        new_phone_count += 1
    else:
        print(row.PHONE)
        print(row.TELEPHONE_NUMBER)
print(f'{yes_count} phones match. {no_count} phones do not match. {new_phone_count} physicians have no ppd phone number, but do have a number on the online roster')

None
nan
None
nan
None
nan
674 phones match. 1305 phones do not match. 944 physicians have no ppd phone number, but do have a number on the online roster


In [70]:
import numpy as np

In [76]:
new_phone_count

944

In [79]:
674+1305+944

2923

In [82]:
mathced_test

,NAME,ME,TELEPHONE_NUMBER,PHONE
0,Youssef Dakka,6.610204e+10,3.139163e+09,800-436-7936
139,Robert Peleman,2.507842e+09,5.862632e+09,586-263-2371
173,Theodore Tangalos,2.507942e+09,5.867265e+09,586-726-4823
214,Anandeep Kumar,4.957091e+10,5.172051e+09,517-205-7525
215,Sarah Money,3.843081e+09,7.349367e+09,800-436-7936
...,...,...,...,...
4921,Fadi Ibrahim,3.050108e+10,2.488800e+09,248-880-0123
5346,Richard Santos,1.803980e+09,NaN,517-205-7060
5358,Christine Meyer,2.501831e+09,2.486669e+09,800-436-7936
5459,William Massey,4.950890e+10,5.867260e+09,586-726-0044
